In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

In [19]:
ind = pd.read_csv("indicators2.0.csv")
ind_drop = ind.drop(["Country Name","Series Code"], axis = 1)
ind_melt = ind_drop.melt(id_vars=['Country Code','Series Name'], var_name= "Year",value_name= "Value")
ind_melt['Year'] = ind_melt['Year'].str.split(' ',expand= True)[0]
indicators = ind_melt.replace("..", np.NaN).dropna()
indicators['Value'] = indicators['Value'].astype(float).round(3)
group = indicators.groupby(['Country Code','Year','Series Name']).mean().reset_index()
Ind = group.pivot(index = ['Country Code','Year'],columns= 'Series Name', values= 'Value').reset_index()
na = Ind.loc[Ind['Country Code'].isin(['CAN','MEX','USA'])]
na['Country Code'] = 'NA'
na.groupby(['Country Code','Year']).mean().round(3)
big_dadE = pd.concat([Ind,na])
big_dadE.loc[big_dadE['Country Code'] == 'OED','Country Code'] = 'OECD'

/var/folders/d9/7cxg74nn7p5fj957krz2nmgw0000gp/T/ipykernel_20395/500759807.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  na['Country Code'] = 'NA'


In [20]:
cci = pd.read_csv("https://richardson.byu.edu/data_contest/CCI.csv")
cci['Year'] = cci['TIME'].str.split("-",expand= True)[0]
cci_drop = cci.drop(['TIME'], axis = 1)
CCI = cci_drop.groupby(['LOCATION','Year']).mean().reset_index()
CCI.rename(columns={'LOCATION': 'Country Code'},inplace= True)

In [21]:
econ = big_dadE.merge(CCI, on = ['Year','Country Code'], how= 'outer')
econ

,Country Code,Year,Consumer price index (2010 = 100),GDP per capita (current US$),GDP per capita growth (annual %),"Inflation, consumer prices (annual %)",Lending interest rate (%),"Unemployment, total (% of total labor force) (national estimate)",CCI
0,CAN,1960,13.344,2259.251,NaN,1.359,5.750,NaN,NaN
1,CAN,1961,13.480,2240.433,1.121,1.019,5.604,NaN,NaN
2,CAN,1962,13.623,2268.585,5.446,1.062,5.708,NaN,NaN
3,CAN,1963,13.845,2374.498,3.405,1.628,5.750,NaN,NaN
4,CAN,1964,14.110,2555.111,4.651,1.912,5.750,NaN,NaN
...,...,...,...,...,...,...,...,...,...
494,NA,2021,154.677,10045.681,4.136,5.689,4.892,4.09,NaN
495,NA,2021,124.266,70248.629,5.820,4.698,3.250,5.35,NaN
496,JPN,2022,NaN,NaN,NaN,NaN,NaN,NaN,97.490043
497,OECD,2022,NaN,NaN,NaN,NaN,NaN,NaN,97.283538


In [23]:
games = pd.read_csv("vgsales.csv")
games['Year'] = pd.to_datetime(games['Year'], format = '%Y').dt.strftime('%Y')
games

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [29]:
all = econ.merge(games, on = 'Year')

all.to_csv('case_comp.csv')
econ.to_csv('case_comp_econ.csv')
games.to_csv('case_comp_games.csv')